In [2]:
import os
import sqlite3
import pandas as pd
import numpy as np
import datetime
from pymystem3 import Mystem
import re
from nltk.corpus import stopwords

conn = sqlite3.connect(r"../../diplom.db")
cur = conn.cursor()

res = cur.execute("""SELECT ID, DDATE, TRANSCRIPT, DEMOCRACY_COUNT FROM NG ORDER BY DEMOCRACY_COUNT DESC LIMIT 25;""")
for i, date, text, count in res:
    print(i, date, count)
    with open(f'{i}.txt', 'w', encoding='utf-8') as f:
        f.write(text)

1699070 2017-11-28 00:14:00 33
1681318 2000-08-17 00:00:00 29
1697143 2009-02-10 00:00:00 29
1678864 2018-08-01 23:51:00 19
1707047 2018-08-01 23:51:00 19
1711464 2010-11-09 00:00:00 17
1679643 2006-08-31 00:00:00 15
1682755 2000-03-08 00:00:00 13
1703335 2004-05-24 00:00:00 13
1679114 2013-04-18 00:01:00 12
1685444 2010-11-16 00:00:00 12
1693457 2009-04-13 00:00:00 12
1697488 2009-09-15 00:00:00 12
1708178 2012-11-13 00:00:00 12
1711184 2008-03-06 00:00:00 12
1711754 2012-04-24 00:00:00 12
1688698 2002-02-18 00:00:00 11
1694135 2014-10-13 21:25:00 11
1691344 2012-03-13 00:00:00 10
1695300 2005-02-25 00:00:00 10
1700330 2000-01-28 00:00:00 10
1701319 2007-06-14 00:00:00 10
1701995 2005-05-08 00:00:00 10
1705298 2009-09-30 00:00:00 10
1686698 2003-08-12 00:00:00 9
